### 털과 날개가 있느냐를 기준으로 포유류,조류,기타를 구분하는 신경망 모델 만들기

In [14]:
import tensorflow as tf
import numpy as np

기본적인 tensorflow와 벡터와 행렬연산에서 쓰일 numpy(Numerical python)을 import해 줍니다

In [2]:
#[털,날개] 있으면1 없으면0
x_data = np.array([[0,0],[1,0],[1,1],[0,0],[0,0],[0,1]])

학습에 사용할 특징 데이터를 정의합니다.

In [4]:
y_data = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1],
    [1,0,0],
    [1,0,0],
    [0,0,1]
])

학습에 사용할 결과값을 레이블 데이터로 정의합니다.   
**레이블 데이터는 `one-hot encoding`이라는 특수한 형태로 구성합니다.**
>one-hot encoding이란 단어 집합의 크기를 벡터의 차원으로 하고, 표현하고 싶은 단어의 인덱스에 1의 값을 부여하고, 다른 인덱스에는 0을 부여하는 단어의 벡터 표현 방식입니다.   

따라서 문자를 기타(0) = [1.0.0], 포유류(1) = [0,1,0], 조류(2) = [0,0,1]와 같은 벡터로써 표현하는 것입니다.

즉,입력과 결과 데이터를 아래와 같이 정해줍니다.   
[0,0] -> [1,0,0] 기타   
[1,0] -> [0,1,0] 포유류   
[1,1] -> [0,0,1] 조류   
[0,0] -> [1,0,0] 기타   
[0,0] -> [1,0,0] 기타   
[0,1] -> [0,0,1] 조류   

* * *
#### 신경망 모델 구성

In [26]:
X= tf.placeholder(tf.float32)
Y= tf.placeholder(tf.float32)

이제 특징X와 레이블 Y간의 관계를 알아내는 모델을 만듭니다.   
여기서 X,Y에 실측값을 넣기위해 placeholder로 설정합니다.   

In [27]:
W =tf.Variable(tf.random_uniform([2,3],-1.,1.))
b = tf.Variable(tf.zeros([3]))

**가중치 변수 W는 [입력층(특징수), 출력층(레이블 수)]의 행렬 형태로 만들어주고 편향 변수 b는 레이블 수인 3개의 요소를 가진 변수로 설정합니다.**

In [28]:
L = tf.add(tf.matmul(X,W),b)
L = tf.nn.relu(L)

이제 인공신경망을 구성합니다 L = ReLU(X * W + b)

In [29]:
model = tf.nn.softmax(L)

추가로 신경망을 통해 나온 출력값을 `softmax` 함수를 이용하여 결과의 확률형태로 나타내 줍니다.   

In [30]:
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(model),axis=1))

`교차 엔트로피`함수를 사용해 손실함수를 만들어 줍니다.   
교차 엔트로피는 label이 one-hot encoding로 표현될때만 사용가능 합니다.    
- 1.우선 실제 정답 레이블 값인 Y와 예측값인 model에 log를 씌워 곱(**행렬곱이 아닌 일반곱**)합니다.   
   Y는 자신이 표현하는 값를 제외하곤 모두 0 이므로 (예: 조류 = [0,0,1]) 결국 두 곱은 예측값(model: 정답이 나올 확률)에 log를 취한값과 0 (예 [log(0.7),0,0])으로 표현됩니다. 여기서 log를 취해준 이유는 확률이 적을수록 값을 크게 만들고 확률이 1일경우 cost를 0으로 만들기 위해서 입니다.   

- 2.-`tf.reduce_sum(ㅁ, axis = 1)`을 통해 1차원 행렬로 만들어 줍니다. 여기서 - 는 log로 인한 음수값을 양수로 바꾸기 위해 넣습니다.
- 3.`tf.reduce_mean`을 통해 1차원 행렬을 하나의 값(교차 엔트로피값)으로 나타냅니다.
 

* * *
학습시키기

In [31]:
optimizer = tf.train.GradientDescentOp
timizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

Day1에서 배운데로 경사하강법으로 cost 최적화를 진행합니다.

In [32]:
init = tf.global_variables_initializer() //텐서플로의 세션을 초기화합니다.
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step +1) % 10 ==0:
        print(step +1 , sess.run(cost,feed_dict={X: x_data, Y: y_data}))

10 1.1849936
20 1.1790208
30 1.1731282
40 1.1673156
50 1.1615816
60 1.1559255
70 1.1503466
80 1.1448437
90 1.1394167
100 1.134139


앞에서 구성한 특징 데이터(X)와 레이블 데이터(Y)를 이용해 학습을 100번 시킵니다.
또한 학습 도중 10번에 1번씩 cost값을 출력해봅니다.

In [33]:
prediction = tf.argmax(model, axis =1)
target = tf.argmax(Y ,axis =1)
print("예측값", sess.run(prediction,feed_dict={X: x_data}))
print("실제값", sess.run(target, feed_dict={Y:y_data}))

예측값 [0 1 1 0 0 0]
실제값 [0 1 2 0 0 2]


학습이 완료된 뒤 예측값인 model중 확률이 가장 큰 idex값을 나타내주는 `tf.argmax`를 사용해 레이블값을 출력하고
실제값과 비교해봅니다.

In [34]:
is_correct = tf.equal(prediction,target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f'%sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))
sess.close()

정확도: 66.67


아쉽게도 정확도가 그리 높지 못합니다. 이는 학습횟수가 문제가 아니라 신경망이 1층이기에 생기는 한계입니다.   
따라서 우리는 다음으로 심층신경망에대해 공부해봅시다.